# Project 4 - Group 16
Names: Yixuan Huang, Anuj Patel, Yunfan Wang, Renlu Zhang

**Attestation**: All 4 of us agree that we each contributed substantially to the following tasks in our project: selecting our datasets and the questions we wanted to answer, wrangling and cleaning the datasets, writing the scripts to conform our data to our schema and conduct our analysis, and prepare our presentation.

***

## Part 1 - Selection

Our group chose to analyze traffic collisions in New York City. Our dataset, which we found on the NYC open data website https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95, contains time, location, vehicle, cause, and victim data for each collision. Because each row in the dataset represents one traffic collision event, we believe that this data set is at the "transactional" level (a collision is a transaction). We thought this would be an interesting dataset to study as we were curious about what insights we could find about accidents in the most densely populated city in the US. Let's begin by downloading our data and exploring it.

In [1]:
!wget "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv"

--2018-12-10 15:09:08--  https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv’

rows.csv                [     <=>            ] 289.91M  53.4MB/s    in 6.0s    

Last-modified header invalid -- time-stamp ignored.
2018-12-10 15:09:14 (48.7 MB/s) - ‘rows.csv’ saved [303997087]



Let's find out the number of lines in the dataset. When we downloaded and cleaned the data on 11/28/2018, it had 1,385,920 lines. **Note that this number will be different as the NYPD updates the dataset regularly. Because we are using Trifacta for data cleaning, we are unable to create an automated cleaning flow within this notebook that handles the addition of new entries.**

In [2]:
!xsv count rows.csv

1386002


What columns are currently in the dataset? What does the data in them look like?

In [3]:
!csvstat -n rows.csv

  1: DATE
  2: TIME
  3: BOROUGH
  4: ZIP CODE
  5: LATITUDE
  6: LONGITUDE
  7: LOCATION
  8: ON STREET NAME
  9: CROSS STREET NAME
 10: OFF STREET NAME
 11: NUMBER OF PERSONS INJURED
 12: NUMBER OF PERSONS KILLED
 13: NUMBER OF PEDESTRIANS INJURED
 14: NUMBER OF PEDESTRIANS KILLED
 15: NUMBER OF CYCLIST INJURED
 16: NUMBER OF CYCLIST KILLED
 17: NUMBER OF MOTORIST INJURED
 18: NUMBER OF MOTORIST KILLED
 19: CONTRIBUTING FACTOR VEHICLE 1
 20: CONTRIBUTING FACTOR VEHICLE 2
 21: CONTRIBUTING FACTOR VEHICLE 3
 22: CONTRIBUTING FACTOR VEHICLE 4
 23: CONTRIBUTING FACTOR VEHICLE 5
 24: UNIQUE KEY
 25: VEHICLE TYPE CODE 1
 26: VEHICLE TYPE CODE 2
 27: VEHICLE TYPE CODE 3
 28: VEHICLE TYPE CODE 4
 29: VEHICLE TYPE CODE 5


In [4]:
!head -n 5 rows.csv | csvlook

|       DATE |    TIME | BOROUGH   | ZIP CODE | LATITUDE | LONGITUDE | LOCATION                | ON STREET NAME                   | CROSS STREET NAME | OFF STREET NAME | NUMBER OF PERSONS INJURED | NUMBER OF PERSONS KILLED | NUMBER OF PEDESTRIANS INJURED | NUMBER OF PEDESTRIANS KILLED | NUMBER OF CYCLIST INJURED | NUMBER OF CYCLIST KILLED | NUMBER OF MOTORIST INJURED | NUMBER OF MOTORIST KILLED | CONTRIBUTING FACTOR VEHICLE 1  | CONTRIBUTING FACTOR VEHICLE 2  | CONTRIBUTING FACTOR VEHICLE 3 | CONTRIBUTING FACTOR VEHICLE 4 | CONTRIBUTING FACTOR VEHICLE 5 | UNIQUE KEY | VEHICLE TYPE CODE 1                 | VEHICLE TYPE CODE 2                 | VEHICLE TYPE CODE 3 | VEHICLE TYPE CODE 4 | VEHICLE TYPE CODE 5 |
| ---------- | ------- | --------- | -------- | -------- | --------- | ----------------------- | -------------------------------- | ----------------- | --------------- | ------------------------- | ------------------------ | ----------------------------- | --------------------------

Immediately, we know what the facts in this dataset are: the pedestrians/cyclists/motorists/persons injured or killed. For the sake of time, we will only analyze the aggregate persons injured and persons killed.

Of the remaining columns, the ones that interest us the most are the contributing factors and vehicle types. We think that it would be interesting to find out which factors caused the most accidents, injuries, and deaths as well as which vehicle types caused the most accidents, injuries, and deaths. Additionally, we would love to examine whether there are any seasonal/time-related trends in the number of accidents and victims using the date and time columns. Lastly, we are also interested in finding any location patterns to traffic incidents and victims. For the sake of time, we will only study this at the borough level. The specific questions we want to answer are:
* Which vehicle types caused the most injuries and deaths?
* Which contributing factors caused the most injuries and deaths?
* What are the trends in number of accidents, injuries, and deaths at the year, month, weekday, time of day, and hour level?
* In which borough did the most accidents, injuries, and deaths occur? Were there any time-related trends within boroughs?

Some concerns we have about the data are that there will be inconsistencies in the data entry, particularly in the vehicle types and contributing factors columns. While capitalization differences are easy to overcome, we already saw that there are possible values for vehicle types that are not mutually exclusive (ex. "Sedan" and "Taxi"). The conclusions achieved through those columns may be weakened as a result.


***
#### BONUS

For the bonus portion to identify and use an additional data source to complement our analysis, we chose to search for weather data (specifically precipitation or any conditions that affect driving). We found the NOAA portal and were able to download weather data from July 1st, 2012 to December 31st, 2014: https://www.ncdc.noaa.gov/cdo-web/datasets/PRECIP_HLY/locations/CITY:US360019/detail. While we hoped to find a dataset that covered a larger chunk of our collisions data, we will make do with only 1.5 years. Let's download and explore this dataset.

In [5]:
!wget "https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/nyweather_jul1_12-jan1_14.csv"

--2018-12-10 15:09:16--  https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/nyweather_jul1_12-jan1_14.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.107.206
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.107.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444772 (434K) [text/csv]
Saving to: ‘nyweather_jul1_12-jan1_14.csv’

nyweather_jul1_12-j 100%[===================>] 434.35K  --.-KB/s    in 0.01s   

2018-12-10 15:09:16 (38.6 MB/s) - ‘nyweather_jul1_12-jan1_14.csv’ saved [444772/444772]



Let's find out the number of lines in the dataset.

In [6]:
!xsv count nyweather_jul1_12-jan1_14.csv

6930


What columns are currently in the dataset? What does the data in them look like?

In [7]:
!csvstat -n nyweather_jul1_12-jan1_14.csv

  1: STATION
  2: STATION_NAME
  3: DATE
  4: HPCP


In [8]:
!head -n 5 nyweather_jul1_12-jan1_14.csv | csvlook

| STATION     | STATION_NAME               | DATE           | HPCP |
| ----------- | -------------------------- | -------------- | ---- |
| COOP:305801 | NY CITY CENTRAL PARK NY US | 20120701 01:00 | 0.00 |
| COOP:305801 | NY CITY CENTRAL PARK NY US | 20120704 05:00 | 0.00 |
| COOP:305801 | NY CITY CENTRAL PARK NY US | 20120704 06:00 | 0.04 |
| COOP:305801 | NY CITY CENTRAL PARK NY US | 20120704 07:00 | 0.02 |


There are readings from different stations in the New York area. Let's see what possible stations there are.

In [9]:
!csvstat nyweather_jul1_12-jan1_14.csv

  1. "STATION"

	Type of data:          Text
	Contains null values:  False
	Unique values:         9
	Longest value:         11 characters
	Most common values:    COOP:286026 (1918x)
	                       COOP:305803 (1802x)
	                       COOP:305811 (1766x)
	                       COOP:305801 (1299x)
	                       COOP:282768 (37x)

  2. "STATION_NAME"

	Type of data:          Text
	Contains null values:  False
	Unique values:         9
	Longest value:         42 characters
	Most common values:    NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US (1918x)
	                       JFK INTERNATIONAL AIRPORT NY US (1802x)
	                       LA GUARDIA AIRPORT NY US (1766x)
	                       NY CITY CENTRAL PARK NY US (1299x)
	                       ESSEX FELLS SERV BUILDING NJ US (37x)

  3. "DATE"

	Type of data:          Text
	Contains null values:  False
	Unique values:         2468
	Longest value:         14 characters
	Most common values:    20120701 01:00 (9

It seems like weather readings from Newark Aiport and JFK Airport are the most common. While La Guardia and Central Park are closer to the center of New York and more representative of the weather in the city, we decided to use only readings from JFK since we wanted the highest number of readings. We decided not to use Newark despite it having the most readings because it's too far away from the city of New York. Using the HPCP (or hourly precipitation) data in inches, we can study the following question:
* What impact does precipitation have on collisions, injuries, and deaths?

A concern we have for the data is that there are many hours that we do not have precipitation data for. We cannot assume that those hours did not have any precipitation and impute values.

***

## Part 2 - Wrangling

Because the free version of Trifacta Wrangler only accepts datasets no larger than 100MB, we need to split our dataset into 4 (with up to 350K non-header rows in each) for cleaning. For the first fourth of our data, we simply need to get the first 350k rows.

In [10]:
!head -n 350000 rows.csv > nyc_crash1.csv

For the 2nd dataset, we need to first add the header row into the file. Then we can append the 2nd set of 350K rows to the file.

In [11]:
!head -n 1 rows.csv > nyc_crash2.csv

In [12]:
!head -n 700000 rows.csv | tail -n 350000 >> nyc_crash2.csv

Again for the 3rd dataset, we need to first add the header row into the file. Then we can append the 3nd set of 350K rows.

In [13]:
!head -n 1 rows.csv > nyc_crash3.csv

In [14]:
!head -n 1050000 rows.csv | tail -n 350000 >> nyc_crash3.csv

Again for the 4th dataset, we need to first add the header row into the file. Then we can append the 4th set of rows which include anything **but** the first 1.05M (or 3 * 350K) rows.

In [15]:
!head -n 1 rows.csv > nyc_crash4.csv

In [16]:
!tail -n +1050001 rows.csv >> nyc_crash4.csv

Let's find out how many rows are in each file. The sum of all 4 should equal our original count.

In [17]:
!xsv count nyc_crash1.csv

349999


In [18]:
!xsv count nyc_crash2.csv

350000


In [19]:
!xsv count nyc_crash3.csv

349999


In [20]:
!xsv count nyc_crash4.csv

336004


When we downloaded the data on 11/28/2018 and ran this step, the total of all 4 line counts was 1,385,920 lines. Note that this number will be different as the NYPD updates the dataset regularly. In the end, it should match the original downloaded dataset.

We then uploaded our datasets into Trifacta Wrangler where we performed the following cleaning steps:
1. Split the Time column into separate Hour and Minute columns.
2. Create a new calculated column for what weekday does the Date column contain (1 represents Monday).
3. Split the Date column into separate Month, Day, and Year columns.
4. In Trifacta Wrangler, we saw that the sum of the number of pedestrians, cyclists, and motorists injured and killed doesn't match the number of persons injured and killed. Thus we deleted the original number of persons injured and killed columns and recalculated them.
5. We created a Time of Day column based on the Hour column that is 'Morning' if the accident time is between 6am-11:59am, 'Afternoon' if the accident time is between 12pm-5:59pm, 'Evening' if the accident time is between 6pm-9:59pm, and 'Night' if the accident time is before 6am or after 10pm.
6. We removed all the fact and dimension columns we are not using in our analysis. These columns are zip code, latitude, longitude, location, on street, cross street, off street, # pedestrians injured, # pedestrians killed, # cyclists injured, # cyclists killed, # motorists injured, and # motorists killed.

The Trifacta recipe that describes all the wrangling steps is shown below:
```
splitpatterns col: TIME type: on on: ':'
derive type: single value: WEEKDAY(DATE) as: 'Weekday'
splitpatterns col: DATE type: on on: '\/' limit: 2
rename type: manual mapping: [DATE1,'Month']
rename type: manual mapping: [DATE2,'Day']
rename type: manual mapping: [DATE3,'Year']
rename type: manual mapping: [TIME1,'Hour']
rename type: manual mapping: [TIME2,'Minute']
derive type: multiple value: {NUMBER OF PEDESTRIANS KILLED} + ({NUMBER OF CYCLIST KILLED} + {NUMBER OF MOTORIST KILLED}) as: 'PERSONS KILLED RECALC'
derive type: multiple value: {NUMBER OF PEDESTRIANS INJURED} + ({NUMBER OF CYCLIST INJURED} + {NUMBER OF MOTORIST INJURED}) as: 'PERSONS INJURED RECALC'
drop col: {NUMBER OF PERSONS INJURED} action: Drop
drop col: {NUMBER OF PERSONS KILLED} action: Drop
move col: {PERSONS INJURED RECALC} position: after after: {NUMBER OF MOTORIST KILLED}
derive type: single value: CASE([(Hour >= 6) && (Hour <= 11), 'Morning', (Hour >= 12) && (Hour <= 17), 'Afternoon', (Hour >= 18) && (Hour <= 21), 'Evening', 'Night']) as: 'Time of Day'
move col: {Time of Day} position: after after: Minute
drop col: {ZIP CODE}, LATITUDE, LONGITUDE, LOCATION, {ON STREET NAME}, {CROSS STREET NAME}, {OFF STREET NAME}, {NUMBER OF PEDESTRIANS INJURED}, {NUMBER OF PEDESTRIANS KILLED}, {NUMBER OF CYCLIST INJURED}, {NUMBER OF CYCLIST KILLED}, {NUMBER OF MOTORIST INJURED}, {NUMBER OF MOTORIST KILLED} action: Drop
```

After running each of our 4 split datasets through this recipe, we have 4 cleaned datasets. We download them from S3 in the next 4 steps below.

In [21]:
!wget "https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash1.csv"

--2018-12-10 15:09:26--  https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash1.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57115599 (54M) [text/csv]
Saving to: ‘clean_nyc_crash1.csv’

clean_nyc_crash1.cs 100%[===================>]  54.47M  57.1MB/s    in 1.0s    

2018-12-10 15:09:27 (57.1 MB/s) - ‘clean_nyc_crash1.csv’ saved [57115599/57115599]



In [22]:
!wget "https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash2.csv"

--2018-12-10 15:09:27--  https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash2.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57737045 (55M) [text/csv]
Saving to: ‘clean_nyc_crash2.csv’

clean_nyc_crash2.cs 100%[===================>]  55.06M  44.5MB/s    in 1.2s    

2018-12-10 15:09:28 (44.5 MB/s) - ‘clean_nyc_crash2.csv’ saved [57737045/57737045]



In [23]:
!wget "https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash3.csv"

--2018-12-10 15:09:28--  https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash3.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56498141 (54M) [text/csv]
Saving to: ‘clean_nyc_crash3.csv’

clean_nyc_crash3.cs 100%[===================>]  53.88M  60.7MB/s    in 0.9s    

2018-12-10 15:09:29 (60.7 MB/s) - ‘clean_nyc_crash3.csv’ saved [56498141/56498141]



In [24]:
!wget "https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash4.csv"

--2018-12-10 15:09:29--  https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_nyc_crash4.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53273504 (51M) [text/csv]
Saving to: ‘clean_nyc_crash4.csv’

clean_nyc_crash4.cs 100%[===================>]  50.80M  55.1MB/s    in 0.9s    

2018-12-10 15:09:30 (55.1 MB/s) - ‘clean_nyc_crash4.csv’ saved [53273504/53273504]



We merge all 4 split datasets into one file `all_crashes.csv`, making sure not to include the header row for the 2nd-4th files.

In [25]:
!cat clean_nyc_crash1.csv > all_crashes.csv

In [26]:
!tail -n +2 clean_nyc_crash2.csv >> all_crashes.csv

In [27]:
!tail -n +2 clean_nyc_crash3.csv >> all_crashes.csv

In [28]:
!tail -n +2 clean_nyc_crash4.csv >> all_crashes.csv

Because we didn't delete any rows as part of our cleaning/wrangling, verify that the row count is the same as our original, unclean file. When we downloaded the data on 11/28/2018 and ran this cleaning step, there were 1,385,920 lines. Because we are using Trifacta for data cleaning, we are unable to create an automated cleaning flow within this notebook that handles the addition of new entries, so from this step on, our data will have 1,385,920 lines regardless of changes to the original datset.

In [29]:
!xsv count all_crashes.csv

1385920


After verifying that we have all our original rows, let's verify that all the columns we wanted in our cleaned dataset are there.

In [30]:
!csvstat -n all_crashes.csv

  1: Month
  2: Day
  3: Year
  4: Weekday
  5: Hour
  6: Minute
  7: Time of Day
  8: BOROUGH
  9: PERSONS INJURED RECALC
 10: PERSONS KILLED RECALC
 11: CONTRIBUTING FACTOR VEHICLE 1
 12: CONTRIBUTING FACTOR VEHICLE 2
 13: CONTRIBUTING FACTOR VEHICLE 3
 14: CONTRIBUTING FACTOR VEHICLE 4
 15: CONTRIBUTING FACTOR VEHICLE 5
 16: UNIQUE KEY
 17: VEHICLE TYPE CODE 1
 18: VEHICLE TYPE CODE 2
 19: VEHICLE TYPE CODE 3
 20: VEHICLE TYPE CODE 4
 21: VEHICLE TYPE CODE 5


Now we can begin our ETL process by setting up PostgreSQL.

In [31]:
%load_ext sql

In [32]:
!dropdb -U student project4

In [33]:
!createdb -U student project4

In [34]:
%sql postgresql://student@/project4

'Connected: student@project4'

First create the table to accomodate our cleaned CSV file with corresponding columns. After setting up dimension tables, this `collisions` table will become our fact table.

In [35]:
%%sql
DROP TABLE IF EXISTS collisions;

CREATE TABLE collisions (
    month INTEGER NOT NULL,
    day INTEGER NOT NULL,
    year INTEGER NOT NULL,
    weekday INTEGER NOT NULL,
    hour INTEGER NOT NULL,
    minute INTEGER NOT NULL,
    time_of_day VARCHAR(9),
    borough VARCHAR(13),
    persons_inj INTEGER NOT NULL,
    persons_kil INTEGER NOT NULL,
    contrib_factor_vehicle_1 VARCHAR(100),
    contrib_factor_vehicle_2 VARCHAR(100),
    contrib_factor_vehicle_3 VARCHAR(100),
    contrib_factor_vehicle_4 VARCHAR(100),
    contrib_factor_vehicle_5 VARCHAR(100),
    key INTEGER NOT NULL,
    vehicle_type_code_1 VARCHAR(100),
    vehicle_type_code_2 VARCHAR(100),
    vehicle_type_code_3 VARCHAR(100),
    vehicle_type_code_4 VARCHAR(100),
    vehicle_type_code_5 VARCHAR(100),
    PRIMARY KEY(key)
);

 * postgresql://student@/project4
Done.
Done.


[]

Load data into the table from our CSV file.

**Note: you will likely have to change the file path with the location of the CSV file based on your directory setup!**

In [36]:
%%sql
COPY collisions FROM '/home/ubuntu/project-4/all_crashes.csv'
CSV
HEADER;

 * postgresql://student@/project4
1385920 rows affected.


[]

Let's peek at the beginning of the table to see if the loading step had any problems.

In [37]:
%%sql
SELECT * FROM collisions
LIMIT 5;

 * postgresql://student@/project4
5 rows affected.


month,day,year,weekday,hour,minute,time_of_day,borough,persons_inj,persons_kil,contrib_factor_vehicle_1,contrib_factor_vehicle_2,contrib_factor_vehicle_3,contrib_factor_vehicle_4,contrib_factor_vehicle_5,key,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
11,16,2018,5,0,10,Night,MANHATTAN,0,0,Unspecified,Unspecified,,,,4021880,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,,,
11,16,2018,5,0,40,Night,,0,0,Pavement Slippery,Pavement Slippery,,,,4021727,Sedan,Pick-up Truck,,,
11,16,2018,5,0,45,Night,BROOKLYN,4,0,Driver Inattention/Distraction,Driver Inattention/Distraction,,,,4021791,Taxi,Taxi,,,
11,16,2018,5,1,0,Night,BRONX,0,0,Pavement Slippery,Unspecified,,,,4021878,Garbage or Refuse,Station Wagon/Sport Utility Vehicle,,,
11,16,2018,5,1,0,Night,BROOKLYN,0,0,Pavement Slippery,Unspecified,,,,4021848,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,,,


Now that we have our to-be fact table `collisions` setup, we need to separate our dimensions and conform all of them to a schema we designed:
![](https://drive.google.com/uc?id=17o7ZWE1ROU5cAsgeDk2fxNLHDSWYWb5s)

Let's begin creating our dimension tables, starting with the collision location context descriptors. Because our analysis only uses the borough of the accident, that will be the only column in this table. In other situations, other columns like zip code or lat/long coordinates would exist in this table. Note the `collision_location_key` column that will contain a serial key for each unique borough.

In [38]:
%%sql
DROP TABLE IF EXISTS collision_location;

CREATE TABLE collision_location (
    collision_location_key SERIAL PRIMARY KEY,
    borough VARCHAR(20)
);

 * postgresql://student@/project4
Done.
Done.


[]

Insert distinct boroughs from `collisions` into this table.

In [39]:
%%sql
INSERT INTO collision_location (borough)
SELECT DISTINCT borough
FROM collisions;

 * postgresql://student@/project4
6 rows affected.


[]

We know that NYC only has 5 boroughs: Manhattan, Brooklyn, Queens, Bronx, and Staten Island. Why were 6 rows affected?

In [40]:
%%sql
SELECT * FROM collision_location;

 * postgresql://student@/project4
6 rows affected.


collision_location_key,borough
1,MANHATTAN
2,BROOKLYN
3,BRONX
4,
5,QUEENS
6,STATEN ISLAND


Some collisions do not have borough information as part of the data. This might be because the accidents happened on the bridges/roads between boroughs (ex. Brooklyn Bridge) or the officer that made the collision report did not enter the borough. This is an important caveat of some of our later location analysis.

Next, we add a column for the location foreign key in our fact table `collisions`.

In [41]:
%%sql
ALTER TABLE collisions
ADD COLUMN collision_location_key INTEGER,
ADD CONSTRAINT fk_collision_location_key
    FOREIGN KEY (collision_location_key)
    REFERENCES collision_location (collision_location_key);

 * postgresql://student@/project4
Done.


[]

And we update the initially NULL entries in that column with the corresponding collision location key.

In [42]:
%%sql
UPDATE collisions
SET collision_location_key = collision_location.collision_location_key
FROM collision_location
WHERE collision_location.borough = collisions.borough;

 * postgresql://student@/project4
1385920 rows affected.


[]

We see that all 1,385,920 (the row size of the cleaned file we downloaded) rows have been affected. Let's make sure all of those rows are not-null.

In [43]:
%%sql
SELECT COUNT(*) num_null_collision_location_keys
FROM collisions
WHERE collision_location_key IS NULL;

 * postgresql://student@/project4
1 rows affected.


num_null_collision_location_keys
0


We have verified that all collision_location_keys in the fact table have values.
***
We can now repeat the same steps for the dimension table for the collision time descriptors. First create the table with all the columns plus a serial key.

In [44]:
%%sql
DROP TABLE IF EXISTS time;

CREATE TABLE time (
    time_key SERIAL PRIMARY KEY,
    month INTEGER NOT NULL,
    day INTEGER NOT NULL,
    year INTEGER NOT NULL,
    weekday INTEGER NOT NULL,
    hour INTEGER NOT NULL,
    minute INTEGER NOT NULL,
    time_of_day VARCHAR(9)
);

 * postgresql://student@/project4
Done.
Done.


[]

Insert distinct combinations of our data columns from `collisions`.

In [45]:
%%sql
INSERT INTO time (month, day, year, weekday, hour, minute, time_of_day)
SELECT DISTINCT month, day, year, weekday, hour, minute, time_of_day
FROM collisions;

 * postgresql://student@/project4
656776 rows affected.


[]

Because we are dealing with a lot more rows than locations, let's create an index to speed up the ETL process.

In [46]:
%%sql
DROP INDEX IF EXISTS idx_time;

CREATE INDEX idx_time ON time (month, day, year, weekday, hour, minute, time_of_day);

 * postgresql://student@/project4
Done.
Done.


[]

Add a column in `collisions` for our time foreign key.

In [47]:
%%sql
ALTER TABLE collisions
ADD COLUMN time_key INTEGER,
ADD CONSTRAINT fk_time_key
    FOREIGN KEY (time_key)
    REFERENCES time (time_key);

 * postgresql://student@/project4
Done.


[]

Update the initially NULL time key to the corresponding key value in the `time` dimension table

In [48]:
%%sql
UPDATE collisions
SET time_key = time.time_key
FROM time
WHERE time.month = collisions.month
AND time.day = collisions.day
AND time.year = collisions.year
AND time.weekday = collisions.weekday
AND time.hour = collisions.hour
AND time.minute = collisions.minute
AND time.time_of_day = collisions.time_of_day;

 * postgresql://student@/project4
1385920 rows affected.


[]

All rows have been affected, but let's verify that none of the key values are still NULL.

In [49]:
%%sql
SELECT COUNT(*) num_null_time_keys
FROM collisions
WHERE time_key IS NULL;

 * postgresql://student@/project4
1 rows affected.


num_null_time_keys
0


Now all time keys in our fact table have values.
***
Let's repeat the same process as above but for the contributing factors for the collision. First create the table with a serial key.

In [50]:
%%sql
DROP TABLE IF EXISTS contributing_factors;

CREATE TABLE contributing_factors (
    factor_key SERIAL PRIMARY KEY,
    factor1 VARCHAR(100),
    factor2 VARCHAR(100),
    factor3 VARCHAR(100),
    factor4 VARCHAR(100),
    factor5 VARCHAR(100)
);

 * postgresql://student@/project4
Done.
Done.


[]

Insert distinct combinations of contributing factors from our fact table `collisions`.

In [51]:
%%sql
INSERT INTO contributing_factors (factor1, factor2, factor3, factor4, factor5)
SELECT DISTINCT contrib_factor_vehicle_1, contrib_factor_vehicle_2, contrib_factor_vehicle_3, contrib_factor_vehicle_4, contrib_factor_vehicle_5
FROM collisions;

 * postgresql://student@/project4
4520 rows affected.


[]

Create an index to speed up the remaining ETL steps.

In [52]:
%%sql
DROP INDEX IF EXISTS idx_factors;

CREATE INDEX idx_factors ON contributing_factors (factor1, factor2, factor3, factor4, factor5);

 * postgresql://student@/project4
Done.
Done.


[]

Add a column into the fact table `collisions` to house the contributing factor foreign key.

In [53]:
%%sql
ALTER TABLE collisions
ADD COLUMN factor_key INTEGER,
ADD CONSTRAINT fk_factor_key
    FOREIGN KEY (factor_key)
    REFERENCES contributing_factors (factor_key);

 * postgresql://student@/project4
Done.


[]

Update the initially NULL values to the corresponding key in the contributing factor dimension table.

In [54]:
%%sql
UPDATE collisions
SET factor_key = contributing_factors.factor_key
FROM contributing_factors
WHERE contributing_factors.factor1 = collisions.contrib_factor_vehicle_1
AND contributing_factors.factor2 = collisions.contrib_factor_vehicle_2
AND contributing_factors.factor3 = collisions.contrib_factor_vehicle_3
AND contributing_factors.factor4 = collisions.contrib_factor_vehicle_4
AND contributing_factors.factor5 = collisions.contrib_factor_vehicle_5;

 * postgresql://student@/project4
1385920 rows affected.


[]

All rows were affected, but let's verify that no contributing factor keys in the fact table are NULL.

In [55]:
%%sql
SELECT COUNT(*) num_null_factor_keys
FROM collisions
WHERE factor_key IS NULL;

 * postgresql://student@/project4
1 rows affected.


num_null_factor_keys
0


All contributing factor keys in our fact table have values.
***
Repeat the same process as above for the vehicle types involved in each collision. First create the table with a serial key.

In [56]:
%%sql
DROP TABLE IF EXISTS vehicle_types;

CREATE TABLE vehicle_types (
    vehicles_key SERIAL PRIMARY KEY,
    vehicle1 VARCHAR(100),
    vehicle2 VARCHAR(100),
    vehicle3 VARCHAR(100),
    vehicle4 VARCHAR(100),
    vehicle5 VARCHAR(100)
);

 * postgresql://student@/project4
Done.
Done.


[]

Insert distinct combinations of vehicle types from our fact table `collisions`.

In [57]:
%%sql
INSERT INTO vehicle_types (vehicle1, vehicle2, vehicle3, vehicle4, vehicle5)
SELECT DISTINCT vehicle_type_code_1, vehicle_type_code_2, vehicle_type_code_3, vehicle_type_code_4, vehicle_type_code_5
FROM collisions;

 * postgresql://student@/project4
7433 rows affected.


[]

Create an index to speed up the remaining ETL steps.

In [58]:
%%sql
DROP INDEX IF EXISTS idx_vehicles;

CREATE INDEX idx_vehicles ON vehicle_types (vehicle1, vehicle2, vehicle3, vehicle4, vehicle5);

 * postgresql://student@/project4
Done.
Done.


[]

Add a column into the fact table `collisions` to house the vehicle types foreign key.

In [59]:
%%sql
ALTER TABLE collisions
ADD COLUMN vehicles_key INTEGER,
ADD CONSTRAINT fk_vehicles_key
    FOREIGN KEY (vehicles_key)
    REFERENCES vehicle_types (vehicles_key);

 * postgresql://student@/project4
Done.


[]

Update the initially NULL values to the corresponding key in the vehicle types dimension table.

In [60]:
%%sql
UPDATE collisions
SET vehicles_key = vehicle_types.vehicles_key
FROM vehicle_types
WHERE vehicle_types.vehicle1 = collisions.vehicle_type_code_1
AND vehicle_types.vehicle2 = collisions.vehicle_type_code_2
AND vehicle_types.vehicle3 = collisions.vehicle_type_code_3
AND vehicle_types.vehicle4 = collisions.vehicle_type_code_4
AND vehicle_types.vehicle5 = collisions.vehicle_type_code_5;

 * postgresql://student@/project4
1385920 rows affected.


[]

All rows were affected, but let's verify that no vehicle type keys in the fact table are NULL.

In [61]:
%%sql
SELECT COUNT(*) num_null_vehicles_keys
FROM collisions
WHERE vehicles_key IS NULL;

 * postgresql://student@/project4
1 rows affected.


num_null_vehicles_keys
0


All vehicle type keys in our fact table have values.
***
#### Bonus
We use our bonus weather data to augment our schema. We first uploaded our dataset into Trifacta Wrangler where we performed the following cleaning steps:
1. Filter to only keep rows from the JFK airport weather station.
2. Split the Date column into separate month, day, year, and hour columns.
3. We removed all the fact and dimension columns we are not using in our analysis. We only keep our date and time columns as well as `HPCP` or hourly precipitation.

The Trifacta recipe that describes all the wrangling steps is shown below:
```
filter type: custom rowType: single row: STATION_NAME == 'JFK INTERNATIONAL AIRPORT NY US' action: Keep
splitpositions col: DATE type: positions positions: 4,6,8 limit: 1
settype col: DATE2 type: Integer
settype col: DATE1 type: Integer
textformat col: DATE4 type: trimwhitespace
extractpatterns type: custom col: DATE4 end: ':'
drop col: DATE4 action: Drop
rename type: manual mapping: [DATE1,'year']
rename type: manual mapping: [DATE2,'month']
rename type: manual mapping: [DATE3,'day']
rename type: manual mapping: [DATE5,'hour']
drop col: STATION,STATION_NAME action: Drop
```

After running our dataset through this recipe, we have a cleaned weather dataset. We download the cleaned file from S3 in the step below.

In [62]:
!wget "https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_weather.csv"

--2018-12-10 15:14:17--  https://s3.amazonaws.com/gwu-workshop-patel-2018/data-management-project-4/clean_weather.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.132.189
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.132.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52293 (51K) [text/csv]
Saving to: ‘clean_weather.csv’

clean_weather.csv   100%[===================>]  51.07K  --.-KB/s    in 0.002s  

2018-12-10 15:14:17 (30.3 MB/s) - ‘clean_weather.csv’ saved [52293/52293]



Repeat the ETL process for the weather data, linking them to collisions by year, month, day, and hour. First create a table with all the columns from our data file plus a serial primary key.

In [63]:
%%sql
DROP TABLE IF EXISTS weather;

CREATE TABLE weather (
    weather_key SERIAL PRIMARY KEY,
    year INTEGER NOT NULL,
    month INTEGER NOT NULL,
    day INTEGER NOT NULL,
    hour INTEGER NOT NULL,
    precip NUMERIC NOT NULL
);

 * postgresql://student@/project4
Done.
Done.


[]

Load data into the table from our CSV file.

**Note: you will likely have to change the file path with the location of the CSV file based on your directory setup!**

In [64]:
%%sql
COPY weather (year, month, day, hour, precip) FROM '/home/ubuntu/project-4/clean_weather.csv'
CSV
HEADER;

 * postgresql://student@/project4
1802 rows affected.


[]

Create an index to speed up the remaining ETL steps.

In [65]:
%%sql
DROP INDEX IF EXISTS idx_weather;

CREATE INDEX idx_weather ON weather (year, month, day, hour, precip);

 * postgresql://student@/project4
Done.
Done.


[]

Add a column into the fact table `collisions` to house the weather foreign key.

In [66]:
%%sql
ALTER TABLE collisions
ADD COLUMN weather_key INTEGER,
ADD CONSTRAINT fk_weather_key
    FOREIGN KEY (weather_key)
    REFERENCES weather (weather_key);

 * postgresql://student@/project4
Done.


[]

Update the initially NULL values to the corresponding key in the vehicle types dimension table.

In [67]:
%%sql
UPDATE collisions
SET weather_key = weather.weather_key
FROM weather
WHERE weather.year = collisions.year
AND weather.month = collisions.month
AND weather.day = collisions.day
AND weather.hour = collisions.hour;

 * postgresql://student@/project4
47631 rows affected.


[]

We know that we only have some of the weather data so most of the weather keys in the fact table will be NULL.

In [68]:
%%sql
SELECT COUNT(*) num_null_weather_keys
FROM collisions
WHERE weather_key IS NULL;

 * postgresql://student@/project4
1 rows affected.


num_null_weather_keys
1338289


As we carry on with our analysis, keep note of this caveat that we do not have a comprehensive weather dataset.
***
Now that we have our separate dimension tables, let's drop our dimensions from our fact table. The only remaining columns within our fact table will now be the fact table's primary key, our 2 facts (`persons_inj` and `persons_kil`), and 5 foreign keys referencing each of the 5 dimension tables.

In [69]:
%%sql
ALTER TABLE collisions
DROP COLUMN month,
DROP COLUMN day,
DROP COLUMN year,
DROP COLUMN weekday,
DROP COLUMN hour,
DROP COLUMN minute,
DROP COLUMN time_of_day,
DROP COLUMN borough,
DROP COLUMN contrib_factor_vehicle_1,
DROP COLUMN contrib_factor_vehicle_2,
DROP COLUMN contrib_factor_vehicle_3,
DROP COLUMN contrib_factor_vehicle_4,
DROP COLUMN contrib_factor_vehicle_5,
DROP COLUMN key,
DROP COLUMN vehicle_type_code_1,
DROP COLUMN vehicle_type_code_2,
DROP COLUMN vehicle_type_code_3,
DROP COLUMN vehicle_type_code_4,
DROP COLUMN vehicle_type_code_5;

 * postgresql://student@/project4
Done.


[]

Let's verify that our fact table conforms to the defined schema and that the number of records in the fact table match our initial file.

In [70]:
%%sql
SELECT * FROM collisions LIMIT 5;

 * postgresql://student@/project4
5 rows affected.


persons_inj,persons_kil,collision_location_key,time_key,factor_key,vehicles_key,weather_key
1,0,4,212286,1559,4094,None
0,0,3,50913,1,1,None
0,0,5,145311,4408,4094,None
0,0,2,274947,4408,4094,None
0,0,4,587043,4408,4094,None


In [71]:
%%sql
SELECT COUNT(*) FROM collisions;

 * postgresql://student@/project4
1 rows affected.


count
1385920


Now that we have verified both the conformity of our schema and the integrity of the rows in our fact table, we can move on to the next step: analysis.

***
## Part 3 - Analysis

Now that our data conforms to our defined schema, we will begin to analyze our data to try to answer the questions we identified in part 1 and any follow-up questions we came up with during our analysis.

#### Which vehicle types caused the most injuries and deaths?

We wrote a SQL script that ran a group by and sum aggregate on all 5 vehicle type columns. After unioning them all together, we can aggregate those once more to find how many injuries were caused by each type of vehicle. The following table shows the top 5 most injury-causing vehicles along with the number of collisions and number of deaths.

In [72]:
%%sql
SELECT LOWER(V.vehicle) AS Vehicle_type, sum(V.collision_count) AS Total_collision, sum(V.injury) AS Total_injury, sum(V.killed) AS Total_killed
FROM (
    SELECT vehicle_types.vehicle1 vehicle, count(vehicle_types.vehicle1) collision_count, sum(persons_inj) injury, sum(persons_kil) killed
    FROM collisions
    JOIN vehicle_types ON collisions.vehicles_key = vehicle_types.vehicles_key
    GROUP BY vehicle_types.vehicle1
    
    UNION ALL
    
    SELECT vehicle_types.vehicle2, count(vehicle_types.vehicle2), sum(persons_inj), sum(persons_kil) killed
    FROM collisions
    JOIN vehicle_types ON collisions.vehicles_key = vehicle_types.vehicles_key
    GROUP BY vehicle_types.vehicle2
    
    UNION ALL
    
    SELECT vehicle_types.vehicle3, count(vehicle_types.vehicle3), sum(persons_inj), sum(persons_kil) killed
    FROM collisions
    JOIN vehicle_types ON collisions.vehicles_key = vehicle_types.vehicles_key
    GROUP BY vehicle_types.vehicle3
    
    UNION ALL
    
    SELECT vehicle_types.vehicle4, count(vehicle_types.vehicle4), sum(persons_inj), sum(persons_kil) killed
    FROM collisions
    JOIN vehicle_types ON collisions.vehicles_key = vehicle_types.vehicles_key
    GROUP BY vehicle_types.vehicle4
    
    UNION ALL
    
    SELECT vehicle_types.vehicle5, count(vehicle_types.vehicle5), sum(persons_inj), sum(persons_kil) killed
    FROM collisions
    JOIN vehicle_types ON collisions.vehicles_key = vehicle_types.vehicles_key
    GROUP BY vehicle_types.vehicle5
) AS V

GROUP BY LOWER(V.vehicle)
HAVING LOWER(V.vehicle) != ''
AND LOWER(V.vehicle) != 'unknown'

ORDER BY sum(V.collision_count) DESC
LIMIT 5;

 * postgresql://student@/project4
5 rows affected.


vehicle_type,total_collision,total_injury,total_killed
passenger vehicle,1346647,367420,1049
sport utility / station wagon,602021,160693,622
taxi,109017,26067,64
sedan,104918,31296,64
station wagon/sport utility vehicle,81990,23294,84


![](https://drive.google.com/uc?id=15tSfR6NUQv6WoueYqe7UjjGZP5TNvpMp)

It is unsurprising that passenger vehicles are involved in the most collisions, injuries, and deaths. They constitute most of the vehicles one sees in the street of NYC. Given that the classifications of vehicle types are not mutually exclusive, as mentioned in our concerns earlier, the conclusions one can gain from this analysis are limited.

***

#### Which contributing factors caused the most injuries and deaths?

Similar to the vehicle types analysis, we wrote a SQL script that ran a group by and sum aggregate on all 5 contributing factor columns. After unioning them all together, we can aggregate those once more to find how many injuries were caused by each factor. The following table shows the top 10 most injury-causing factors along with the number of collisions and number of deaths.

In [73]:
%%sql
SELECT LOWER(F.factor) AS Contributing_Factors, sum(F.collision_count) AS Total_collision, sum(F.injured) AS Total_injury, sum(F.killed) AS Total_killed
FROM(
    SELECT contributing_factors.factor1 factor, count(contributing_factors.factor1) collision_count, sum(persons_inj) injured, sum(persons_kil) killed
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor1
    
    UNION ALL
    
    SELECT contributing_factors.factor2, count(contributing_factors.factor2) collision_count, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor2
    
    UNION ALL
    
    SELECT contributing_factors.factor3, count(contributing_factors.factor3) collision_count, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor3
    
    UNION ALL
    
    SELECT contributing_factors.factor4, count(contributing_factors.factor4) collision_count, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor4
    
    UNION ALL
    
    SELECT contributing_factors.factor5, count(contributing_factors.factor5) collision_count, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor5
) AS F

GROUP BY F.factor
HAVING F.factor != 'Unspecified'
AND F.factor != ''
ORDER BY sum(F.collision_count) DESC
LIMIT 10;

 * postgresql://student@/project4
10 rows affected.


contributing_factors,total_collision,total_injury,total_killed
driver inattention/distraction,282174,74528,208
failure to yield right-of-way,82874,33611,140
other vehicular,66486,14418,31
fatigued/drowsy,63126,17514,3
following too closely,58204,19661,20
backing unsafely,53640,5473,27
turning improperly,41474,6415,11
passing or lane usage improper,28988,3843,12
lost consciousness,26851,3350,22
traffic control disregarded,24176,14816,170


![](https://drive.google.com/uc?id=1yG5T24xSw1p1gFScpex0ppJPeP5pIbzD)

It is unsurprising that driver innatention/distraction caused the most collisions, injuries, and deaths. More surprising is the the disproportional injury and death rate associated with disregarding traffic control! With about the same number of collisions cause as losing consciousness, disregarding traffic control causes about 4 times as many injuries and about 8 times as many deaths!

***

#### What are the trends in number of accidents, injuries, and deaths at the year, month, weekday, time of day, and hour level?

To analyze any trends at the year level, we wrote a SQL script that grouped by the collision year. We excluded the years 2012 and 2018 because we only have partial data for those years.

In [74]:
%%sql
SELECT time.year, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people
FROM collisions
JOIN time ON collisions.time_key = time.time_key
WHERE time.year != 2012 AND time.year != 2018
GROUP BY time.year
ORDER BY time.year;

 * postgresql://student@/project4
5 rows affected.


year,total_collision,injured_people,killed_people
2013,203724,55121,297
2014,206028,51207,262
2015,217691,51355,243
2016,228282,60374,240
2017,229423,61078,254


![](https://drive.google.com/uc?id=1XfHUm0ywg1VhimQm7oIBs4dUaWpiO15W)

Seems like traffic collisions and injuries have increased over this time period. But we're curious to see how this trend compares to the volume of vehicles in the city.

Let's examine the collisions on a month level. Because we're looking at the aggregate sums, we once again exclude 2012 and 2018.

In [75]:
%%sql
SELECT time.month, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people
FROM collisions
JOIN time ON collisions.time_key = time.time_key
WHERE time.year != 2012 AND time.year != 2018
GROUP BY time.month
ORDER BY time.month;

 * postgresql://student@/project4
12 rows affected.


month,total_collision,injured_people,killed_people
1,83995,20518,103
2,77542,17089,81
3,88316,21219,83
4,85895,22104,97
5,97018,25430,105
6,96162,25363,112
7,93179,24980,126
8,91473,25218,131
9,92211,25333,112
10,95488,25341,115


We can see that the number of collisions decreases in the post-Christmas winter months but peaks in late spring/early summer. This might be because less people are likely to drive in snowy conditions and more people are likely driving in optimal weather conditions. It could also be related to the number of tourists in the city (more in May/June and less in January/February). Injury numbers follow the same trend. Collision-related deaths, while they dip in the winter just like collisions and injuries, seem to peak in August and November instead.

Let's examine the collisions at the weekday level.

In [76]:
%%sql
SELECT time.weekday, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people
FROM collisions
JOIN time ON collisions.time_key = time.time_key
GROUP BY time.weekday
ORDER BY time.weekday;

 * postgresql://student@/project4
7 rows affected.


weekday,total_collision,injured_people,killed_people
1,198337,50306,249
2,206044,51333,216
3,204379,50884,210
4,208683,51462,210
5,221574,55466,242
6,183825,51552,241
7,163078,48781,256


Knowing that 1 is Monday, we see that the number of collisions is somewhat higher on weekdays, with the highest peak on Friday. Injuries follow a similar pattern but without as much of a decrease on Saturdays. Deaths seem to follow a different pattern, with the least deaths in the middle of the week and the most on Friday, Saturday, Sunday, and Monday. We can't think of any explanation for this trend.

Let's examine the collisions at the hour level.

In [77]:
%%sql
SELECT time.hour, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people
FROM collisions
JOIN time ON collisions.time_key = time.time_key
GROUP BY time.hour
ORDER BY time.hour;

 * postgresql://student@/project4
24 rows affected.


hour,total_collision,injured_people,killed_people
0,38695,11591,72
1,21646,6947,57
2,16443,5565,61
3,14128,4983,68
4,16507,6228,94
5,18132,5933,64
6,28705,8332,71
7,39668,11217,39
8,77952,18441,38
9,76894,15927,50


![](https://drive.google.com/uc?id=1UefQg36BP_0eVZooerjBrH0NWdpL6UcB)

We can see that both collisions and injuries peak in the afternoon around 4-5pm. We believe that as drivers cause more accidents and injuries as they anxiously try to get home after a long workday. Interestingly, deaths peaks during this time but *also* slightly peaks around the 4am-6am time. People rushing to get to work is a potential cause of this.

Finally, let's examine the collisions at the time of day level (morning, afternoon, evening, or night).

In [78]:
%%sql
SELECT time.time_of_day, count(*) AS Total_collision, sum(persons_inj) AS Total_injury, sum(persons_kil) AS Total_killed
FROM collisions
JOIN time ON collisions.time_key = time.time_key
GROUP BY time.time_of_day
ORDER BY sum(persons_inj) DESC, sum(persons_kil) DESC;

 * postgresql://student@/project4
4 rows affected.


time_of_day,total_collision,total_injury,total_killed
Afternoon,546013,129707,394
Morning,367586,83480,310
Evening,266789,78284,341
Night,205532,68313,579


Seems like collisions and injuries happen more in the afternoon. Deaths, however, occur more often at night - maybe the darkness has an impact on how lethal collisions are.

***

#### In which borough did the most accidents, injuries, and deaths occur? Were there any time-related trends within boroughs?

To obtain those metrics per borough, we wrote a SQL script that grouped by the borough of the accident. We excluded collisions that had no borough labeled. We think that while it is likely that, in most cases, the officer reporting the collision simply didn't add any location info, some of the unmarked collisions could represent accidents that occured on roads that connect two boroughs (ex. Brooklyn Bridge, which is in between Manhattan and Brooklyn).

In [79]:
%%sql
SELECT collision_location.borough, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people
FROM collisions
JOIN collision_location ON collisions.collision_location_key = collision_location.collision_location_key
WHERE collision_location.borough != ''
GROUP BY collision_location.borough
ORDER BY count(*) DESC;

 * postgresql://student@/project4
5 rows affected.


borough,total_collision,injured_people,killed_people
BROOKLYN,301660,87785,340
QUEENS,257042,67083,318
MANHATTAN,240732,41469,194
BRONX,132687,37775,149
STATEN ISLAND,44368,10689,60


![](https://drive.google.com/uc?id=1cgZBPcX8iMBsSkUY6WLr0ccpuJ3BhX5N)

Seems like most collisions, injuries, and deaths occur in Brooklyn, followed by Queens, Manhattan, Bronx, then Staten Island. This order of boroughs is the same if we were to order them by population, so we think that these metrics might be correlated.

Let's also look at these metrics split by borough over the years. Once again, we are excluding collisions with no borough data and the years 2012 and 2018 because of partial data.

In [80]:
%%sql
SELECT collision_location.borough, time.year, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people
FROM collisions
JOIN collision_location ON collisions.collision_location_key = collision_location.collision_location_key
JOIN time ON collisions.time_key = time.time_key
WHERE time.year != 2012 AND time.year != 2018
AND collision_location.borough != ''
GROUP BY collision_location.borough, time.year
ORDER BY collision_location.borough, time.year;

 * postgresql://student@/project4
25 rows affected.


borough,year,total_collision,injured_people,killed_people
BRONX,2013,19768,6259,32
BRONX,2014,19687,5886,19
BRONX,2015,21257,5626,26
BRONX,2016,21460,5997,20
BRONX,2017,21197,5838,17
BROOKLYN,2013,47017,15410,67
BROOKLYN,2014,47759,14244,67
BROOKLYN,2015,50847,13706,67
BROOKLYN,2016,47157,13404,33
BROOKLYN,2017,44671,12163,40


Seems like most boroughs had increasing year-to-year accidents, injuries, and deaths until about 2015 and 2016. From there, all boroughs began experiencing declines across all three metrics.

***

#### Ad-hoc Analysis

As we were performing the analysis above, we came up with a couple more questions we wanted to answer. First, we wanted to find the time of day and borough that had the most injuries, and compare with the number of collisions and number of deaths.

In [81]:
%%sql
SELECT time.time_of_day, collision_location.borough, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people
FROM collisions
JOIN collision_location ON collisions.collision_location_key = collision_location.collision_location_key
JOIN time ON collisions.time_key = time.time_key
AND collision_location.borough != ''
GROUP BY time.time_of_day, collision_location.borough
ORDER BY sum(persons_inj) DESC;

 * postgresql://student@/project4
20 rows affected.


time_of_day,borough,total_collision,injured_people,killed_people
Afternoon,BROOKLYN,122393,32428,95
Afternoon,QUEENS,101780,24478,82
Evening,BROOKLYN,58189,20102,77
Morning,BROOKLYN,79847,19728,73
Morning,QUEENS,69913,16652,62
Night,BROOKLYN,41231,15527,95
Evening,QUEENS,50319,14938,68
Afternoon,MANHATTAN,92977,14385,70
Afternoon,BRONX,51671,13904,37
Night,QUEENS,35030,11015,106


Brooklyn and Queens in the afternoon seems to be the most dangerous (injury-wise). Brooklyn in the evening is also pretty bad! These once again make sense as these two boroughs have the highest population and afternoon is when people going home after work cause the most accidents.

Let's find which time of day and contributing factor had the most injuries, along with the number of accidents and deaths.

In [82]:
%%sql
SELECT time.time_of_day, F.factor AS Contributing_Factors, sum(collision_num) AS Total_collision, sum(F.injured) AS Total_injury, sum(F.killed) AS Total_killed
FROM(
    SELECT contributing_factors.factor1 factor, count(contributing_factors.factor1) collision_num, collisions.time_key, sum(persons_inj) injured, sum(persons_kil) killed
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor1, collisions.time_key
    
    UNION ALL
    
    SELECT contributing_factors.factor2, count(contributing_factors.factor2), collisions.time_key, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor2, collisions.time_key
    
    UNION ALL
    
    SELECT contributing_factors.factor3, count(contributing_factors.factor3), collisions.time_key, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor3, collisions.time_key
    
    UNION ALL
    
    SELECT contributing_factors.factor4, count(contributing_factors.factor4), collisions.time_key, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor4, collisions.time_key
    
    UNION ALL
    
    SELECT contributing_factors.factor5, count(contributing_factors.factor5), collisions.time_key, sum(persons_inj) injured_people, sum(persons_kil)
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    GROUP BY contributing_factors.factor5, collisions.time_key
) AS F

JOIN time ON time.time_key = F.time_key
GROUP BY time.time_of_day, F.factor
HAVING F.factor != 'Unspecified'
AND F.factor != ''
ORDER BY sum(F.injured) DESC
LIMIT 10;

 * postgresql://student@/project4
10 rows affected.


time_of_day,contributing_factors,total_collision,total_injury,total_killed
Afternoon,Driver Inattention/Distraction,113829,27752,54
Morning,Driver Inattention/Distraction,72262,17326,38
Evening,Driver Inattention/Distraction,56003,16169,38
Night,Driver Inattention/Distraction,40080,13281,78
Afternoon,Failure to Yield Right-of-Way,34140,12575,55
Morning,Failure to Yield Right-of-Way,22109,8396,42
Evening,Failure to Yield Right-of-Way,17492,8235,29
Afternoon,Following Too Closely,24254,7839,5
Afternoon,Fatigued/Drowsy,26319,6995,0
Afternoon,Other Vehicular,27237,5070,13


![](https://drive.google.com/uc?id=1a-XuHo1opKss8UpyO8qHLXPV-PK24QLh)

Seems like no matter what the time of day, driver innattention/distraction causes the most injuries. One thing that stood out to us was that driver inattention/distraction at night leads to proportionally more fatalities than at any other time of day! So drivers should be even more attentive when driving at night.

***

#### BONUS - How does precipitation affect the number of accidents, injuries, and deaths?

We first look at collisions during which we have precipitation data and compare, on the month level, the number of collisions, number of people injured, number of people killed, and average precipitation.

In [83]:
%%sql
SELECT time.month, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people, ROUND(avg(weather.precip),3) avg_precip
FROM collisions
JOIN time ON collisions.time_key = time.time_key
JOIN weather ON collisions.weather_key = weather.weather_key
GROUP BY time.month
ORDER BY time.month;

 * postgresql://student@/project4
12 rows affected.


month,total_collision,injured_people,killed_people,avg_precip
1,2449,721,4,0.023
2,3694,965,6,0.020
3,3920,1011,7,0.019
4,1752,496,1,0.031
5,3628,1013,0,0.025
6,3547,986,5,0.075
7,4056,1201,0,0.033
8,3716,1020,5,0.059
9,3187,1012,7,0.050
10,4101,1223,6,0.021


![](https://drive.google.com/uc?id=1LXOUnks8qhBJA89kG4kSh9ChzxeSwQaw)

Seems like average precipitation levels have no effect on the number of collisions, injuries, or people killed.

Maybe we need to look at a different aggregation level to find an effect. Let's perform the same analysis as above but at the hour level.

In [84]:
%%sql
SELECT time.hour, count(*) AS Total_collision, sum(persons_inj) injured_people, sum(persons_kil) killed_people, ROUND(avg(weather.precip),3) avg_precip
FROM collisions
JOIN time ON collisions.time_key = time.time_key
JOIN weather ON collisions.weather_key = weather.weather_key
GROUP BY time.hour
ORDER BY time.hour;

 * postgresql://student@/project4
24 rows affected.


hour,total_collision,injured_people,killed_people,avg_precip
0,1179,354,4,0.030
1,937,318,5,0.056
2,634,202,1,0.059
3,498,182,3,0.030
4,643,211,1,0.032
5,740,254,2,0.031
6,1042,333,2,0.019
7,1452,457,2,0.013
8,2747,683,0,0.019
9,2511,553,0,0.029


![](https://drive.google.com/uc?id=1G52wQcBfa4Hq4-xMvq_0ypYyI-iOnrpW)

Again, average precipitation levels seem to share no relationship with the number of collisions, injuries, or deaths.

Instead of looking for correlation between collisions and the *level* of precipitation, we decided to find if injury and death rates are different with our without precipitation.

In [85]:
%%sql
SELECT 'Precipitation = 0' precip_level,
    ROUND((1.0 * SUM(collisions.persons_inj)/COUNT(collisions.*)),5) injury_per_accident,
    ROUND((1.0 * SUM(collisions.persons_kil)/COUNT(collisions.*)),5) deaths_per_accident
FROM collisions
JOIN weather ON collisions.weather_key = weather.weather_key
WHERE weather.precip = 0

UNION ALL

SELECT 'Precipitation > 0' precip_level,
    ROUND((1.0 * SUM(collisions.persons_inj)/COUNT(collisions.*)),5) injury_per_accident,
    ROUND((1.0 * SUM(collisions.persons_kil)/COUNT(collisions.*)),5) deaths_per_accident
FROM collisions
JOIN weather ON collisions.weather_key = weather.weather_key
WHERE weather.precip > 0;

 * postgresql://student@/project4
2 rows affected.


precip_level,injury_per_accident,deaths_per_accident
Precipitation = 0,0.28114,0.00135
Precipitation > 0,0.28122,0.00109


Looks like the presence of precipitation doesn't change the rate of injuries per accident and the rate of deaths per accident by a significant amount.

Shifting gears, what are the collision contributing factors associated with higher precipitation?

In [86]:
%%sql
SELECT LOWER(F.factor) AS Contributing_Factors, ROUND(sum(F.precip)/sum(F.collision_count),3) avg_precip
FROM(
    SELECT contributing_factors.factor1 factor, count(contributing_factors.factor1) collision_count, sum(weather.precip) precip
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    JOIN weather ON collisions.weather_key = weather.weather_key
    GROUP BY contributing_factors.factor1
    
    UNION ALL
    
    SELECT contributing_factors.factor2, count(contributing_factors.factor2) collision_count, sum(weather.precip) precip
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    JOIN weather ON collisions.weather_key = weather.weather_key
    GROUP BY contributing_factors.factor2
    
    UNION ALL
    
    SELECT contributing_factors.factor3, count(contributing_factors.factor3) collision_count, sum(weather.precip) precip
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    JOIN weather ON collisions.weather_key = weather.weather_key
    GROUP BY contributing_factors.factor3
    
    UNION ALL
    
    SELECT contributing_factors.factor4, count(contributing_factors.factor4) collision_count, sum(weather.precip) precip
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    JOIN weather ON collisions.weather_key = weather.weather_key
    GROUP BY contributing_factors.factor4
    
    UNION ALL
    
    SELECT contributing_factors.factor5, count(contributing_factors.factor5) collision_count, sum(weather.precip) precip
    FROM collisions
    JOIN contributing_factors ON collisions.factor_key = contributing_factors.factor_key
    JOIN weather ON collisions.weather_key = weather.weather_key
    GROUP BY contributing_factors.factor5
) AS F

GROUP BY F.factor
HAVING F.factor != 'Unspecified'
AND F.factor != ''
ORDER BY sum(F.precip)/sum(F.collision_count) DESC
LIMIT 10;

 * postgresql://student@/project4
10 rows affected.


contributing_factors,avg_precip
shoulders defective/improper,0.113
obstruction/debris,0.056
pavement defective,0.053
traffic control device improper/non-working,0.047
pavement slippery,0.042
other electronic device,0.042
brakes defective,0.041
view obstructed/limited,0.040
windshield inadequate,0.040
drugs (illegal),0.038


![](https://drive.google.com/uc?id=1006Q5FJ4D90mKxNuCxrxvMwTde3d-vwX)

Looks like infrastructure problems like defective shoulders and pavement, and non-working traffic control devices as causes of accidents associated with higher levels of precipitation. This is to be expected as rain and snow are known for putting stress on various elements of a city's transit infrastructure.